In [1]:
import sys

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")
from tudatpy.data.mpc import BatchMPC
from tudatpy.data.horizons import HorizonsQuery

from tudatpy.numerical_simulation import environment_setup
from tudatpy.interface import spice

import numpy as np
import pytest
import datetime
import matplotlib.pyplot as plt

spice.load_standard_kernels()

In [2]:
def test_compare_horizons_spice(pos_tolerance=1e3, vel_tolerance=1e-3):
    """Compares the states from an array of times between HorizonsQuery and SPICE"""
    start = 20 * 365 * 86400
    end = 23 * 365 * 86400

    npoints = 100

    times = np.linspace(start, end, npoints)

    query = HorizonsQuery(
        query_id="299", location="500@399", epoch_list=list(times), extended_query=True
    )
    ref = "ECLIPJ2000"

    horizons_states = query.cartesian(frame_orientation=ref)[:, 1:]
    spice_states = []

    for time in times:
        # from spice
        state_spice = spice.get_body_cartesian_state_at_epoch(
            "Venus", "Earth", ref, "NONE", time
        )
        spice_states.append(state_spice)

    diff = np.abs(horizons_states - spice_states)
    max_diff_pos = diff.max(axis=0)[0:3].max()
    max_diff_vel = diff.max(axis=0)[3:6].max()

    assert max_diff_pos < pos_tolerance
    assert max_diff_vel < vel_tolerance

In [3]:
test_compare_horizons_spice()

In [4]:
def test_compare_mpc_horizons_eph():
    batch = BatchMPC()
    batch.get_observations([433])
    batch.filter(
        epoch_start=datetime.datetime(2017, 1, 1),
        epoch_end=datetime.datetime(2022, 1, 1),
        observatories=["T08"],
    )
    batch_times = batch.table.epochJ2000secondsTDB.to_list()

    eros = HorizonsQuery(
        query_id="433;", location="T08@399", epoch_list=batch_times, extended_query=True
    )

    radec_horizons = eros.interpolated_observations(degrees=False)
    radec_mpc = batch.table.loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]].reset_index(
        drop=True
    )

    diff = (radec_horizons - radec_mpc).to_numpy()
    diff = np.abs(diff).max(axis=0)


    time_diff = diff[0]
    RA_diff = diff[1]
    DEC_diff = diff[2]

    print(time_diff)
    print(RA_diff)
    print(DEC_diff)

    assert time_diff < 1e-3
    assert RA_diff < 1e-5
    assert DEC_diff < 1e-5


test_compare_mpc_horizons_eph()

0.0008889436721801758
1.2541470582050351e-06
1.941696246232638e-06


In [5]:
# query = HorizonsQuery(
#     query_id="439;",
#     location="500@299",
#     epoch_start=datetime.datetime(2022, 1, 1),
#     epoch_end=datetime.datetime(2022, 2, 1),
#     epoch_step="90030",
#     epoch_list=None,
#     extended_query=True,
# )

# print((query.cartesian()).shape)

In [ ]:
def test_jpl_hybrid_function():
    settings = jpl_horizons()